##   `lab10`—Image Manipulation

![](./img/lab09-header-bkgd.png)

❖ Objectives

-   Understand how images are represented and manipulated in memory.

### Obscuring Images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

The moment digital images are created, they are subject to being manipulated, censored, and altered.  This is a source of enormous power for good and ill.  Today we will take a look at how image censorship can be implemented (either to protect individuals' anonymity or to conceal sensitive information).

Several strategies may be used to obscure information such as civilian faces or private data:

- [censor bars](https://infogalactic.com/info/Censor_bars) simply cover the information which needs to be concealed ([famously used by the federal government in released classified documents](http://www.theonion.com/article/cia-realizes-its-been-using-black-highlighters-all-1848))
- [pixelization](https://infogalactic.com/info/Pixelization) censors by dropping the local resolution of the information
- <a href="https://infogalactic.com/info/Fogging_(censorship)">fogging</a> applies a blur effect over an area which yields a smoother appearance than pixelization but has the same effect

<table>
<tr>
<td><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/5d/Pixelization_mosaic.jpg/148px-Pixelization_mosaic.jpg" width="210px;"></td>
<td><img src="./img/fogging-example.png" width="150px;"></td>
</tr>
<tr><td>Censorship via pixelization</td><td>Censorship via fogging</td></tr>
</table>

In this lab, you will implement all three methods as an exercise in image manipulation.

###  Image Data Structure

We will use the `imageio` library function [`imread`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.misc.imread.html) to open an image file as a NumPy array.  Dimensions 0 and 1 of this array represent the 2D layout of the pixels, which are represented either as single values (if greyscale) or a three-element array (if color).

A black-and-white or greyscale image would be represented as follows:

![](tiny-python.png)

In [ ]:
from imageio import imread,imsave

python = imread( 'tiny-python.png' )
plt.imshow( python,cmap="gray" )

plt.show()
print( python )

We will flatten any color images to greyscale to make our subsequent work easier.

In [ ]:
def rgb2gray( rgb ):
    '''
    Convert an array to greyscale.  Code by waspinator.
    '''
    r,g,b = rgb[ :,:,0 ],rgb[ :,:,1 ],rgb[ :,:,2 ]
    gray = 0.2989*r + 0.5870*g + 0.1140*b
    return gray

For representative images, we will use a sequence of pictures of famous whistleblowers:  [Julian Assange](https://en.wikipedia.org/wiki/WikiLeaks), [Edward Snowden](https://en.wikipedia.org/wiki/Edward_Snowden), and [Daniel Ellsberg](https://en.wikipedia.org/wiki/Pentagon_Papers).

Get the dimensions.  For `assange`, the total size can be found using `assange.shape`.  Note that the way they are displayed is _transposed_ relative to the way we think of coördinates:  be careful!

In [ ]:
assange = imread( 'assange.png' )
assange = rgb2gray( assange )

plt.imshow( assange,cmap="gray" )
plt.show()

print( "(#rows, #columns) = %i,%i = (range of y values, range of x values)" % (assange.shape[0],assange.shape[1] ) )

We will need to specify the coordinates of his face, which is centered at `(375,200)` or alternatively the rectangle from `(300,100)` to `(500,320)`.

> Again, due to `imshow`'s transposition of the displayed array, we are not treating $x$ and $y$ coordinates completely consistently below.  Just be careful and you shouldn't have any trouble.

##  Censor Bars

Applying censor bars to an image is quite straightforward, and just involves blacking out the pixel ranges involved (_i.e._, setting the values to zero).

The function you will compose, `censor`, needs to receive the image (`image`) and a range of pixels to censor (`patch`).  `patch` should be a tuple with four values, `( left,top,height,width )`.

<div class="alert alert-warning">
We are not making the implementation robust, so for instance selecting a range outside of the image range will lead to an <code>IndexError</code>.  (This isn't good for production code, but is fine for a quick-and-dirty implementation like this one.)
</div>

-   Compose a function `censor( image,patch )` which accepts these parameters and returns an image `outimage` with the patch replaced by `0`s.  (Note that black for a greyscale image is different from black for a color image!  How?)

In [ ]:
def censor( image,patch ):
    outimage = image.copy()
    black = ???    # black
    
    left,top,height,width = patch
    for w in ???:  # what range using left,width
        for h in ???:  # what range using top,height
            outimage[ h,w ] = black
    return outimage

In [ ]:
assange_censored = censor( assange,( 300,100,220,200 ) )
plt.imshow( assange_censored,cmap="gray" )
plt.show()

##  Pixelization

In [ ]:
snowden = imread( 'snowden.png' )
snowden = rgb2gray( snowden )

plt.imshow( snowden,cmap="gray" )
plt.show()

print( "(#rows, #columns) = %i,%i = (range of y values, range of x values)" % (snowden.shape[0],snowden.shape[1] ) )

We could proceed one of two ways to pixelate the image.  We could select a portion of the image, scale it down (_resample_ it) and then scale that sector back up to size; or we could simply select successive blocks as giant "pixels" and average the value of the pixels within that block.  Libraries such as [Pillow](https://python-pillow.org/) support the first, but we'll implement the second way since it requires more interesting math of you.

What you will do is select each block and average the values inside of it, then share that average value with all of the pixels in that block.  In the graphic below, as each 2×2 block is selected, it is replaced by the average value, resulting finally in the pixelated block at bottom.

![](./img/pixelization.png)

The function you will compose, `pixelate`, needs to receive the image (`image`), a range of pixels to pixelate (`patch`), and a resolution to pixelate to (`res`).  Your result should look like this:

![](./snowden_pixelated.png)

-   Compose a function `pixelate( image,patch,res )` which accepts these parameters and returns an image `outimage` with the patch transformed into `res`$\times$`res`-pixel blocks.  `res` should default to `10`.

In [ ]:
def pixelate( image,patch,res ):  # set the default!
    outimage = image.copy()
    
    left,top,height,width = patch
    xstride = ???  # use top here and think about jumping res steps for the entire height
    ystride = ???  # use left here and think about jumping res steps for the entire width
    
    for i in xstride:
        for j in ystride:
            outimage[ ??? ] = outimage[ ??? ].mean()  # what should the range of each block be?
    return outimage

In [ ]:
# Test your function on the image.
snowden_pixelated = pixelate( snowden,( 280,100,220,200 ),25 )
plt.imshow( snowden_pixelated,cmap="gray" )
plt.show()

##  Fogging

In [ ]:
ellsberg = imread( 'ellsberg.png' )
ellsberg = rgb2gray( ellsberg )

plt.imshow( ellsberg,cmap="gray" )
plt.show()

print( "(#rows, #columns) = %i,%i = (range of y values, range of x values)" % (ellsberg.shape[0],ellsberg.shape[1] ) )

To blur the image, you need to use a function to average together neighboring pixels in order to reduce detail.  One simple way to achieve this is the [box blur](), in which the average of neighboring pixels in a sliding block are used to determine the pixel's new value.

    255 255 245
    128 110  96   →   121 (at the center)
      0   0   0

Other popular filters which one often sees in image processing applications like Adobe Photoshop or GIMP include the [gaussian blur](https://infogalactic.com/info/Gaussian_blur) and the circular box blur.

The function you will compose, `fog`, needs to receive the image (`image`), a range of pixels to pixelate (`patch`), and a "radius" to blur (`radius`, really the limits of the sliding box).

-   Compose a function `fog( image,patch,radius )` which accepts these parameters and returns an image `outimage` with a patch blurred.  `radius` should default to 5.

In [ ]:
def fog( image,patch,radius=5 ):
    outimage = image.copy()
    
    left,top,height,width = patch
    
    for w in ???:       # should go one pixel at a time and use left,width
        for h in ???:   # should go one pixel at a time and use top,height
            outimage[ h,w ] = ???  # should be the mean of the surrounding box of perimeter 2*radius, based on image, not outimage---why?
    return outimage

In [ ]:
# Test your function on the image.
ellsberg_fogged = fog( ellsberg,( 200,100,200,200 ) )
plt.imshow( ellsberg_fogged,cmap="gray" )
plt.show()

That looks pretty good, but the edges of the patch are sharp and somewhat jarring.  Let's improve on that.

Finally, you will create a function which is itself smoothed out, so that the edges of the censored patch blend seamlessly with the surrounding image context.  We will make two changes in our approach to facilitate this:

1.  Change from a box patch to a circular patch.
2.  Vary the radius of the box blur from 1 at the edge to a maximum near the center.  We can use the same logic as the `fog` function to accomplish this.

The function you will compose, `smoothfog`, needs to receive the image (`image`) and the center of the circle and its radius (`center`, a three-member tuple).

-   Compose a function `smoothfog( image,center )` which accepts these parameters and returns an image `outimage` with a patch blurred smoothly.

In [ ]:
from scipy.spatial.distance import euclidean as dist

def smoothfog( image,center ):
    outimage = image.copy()
    
    cirx,ciry,cirr = center
    
    for w in ???:       # should use cirr and cirx to define x boundaries
        for h in ???:   # should use cirr and ciry to define y boundaries
            radius = dist( np.array( ( w,h ) ),np.array( ( cirx,ciry ) ) )
            if radius > ???:  # and filter out the pixels outside the circle
                continue  # don't alter pixels outside the circle
            rad = ???  # set `rad`, the distance of the pixel from the edge of the circle, as an int
            outimage[ h,w ] = ???  # set as with `fog` but using `rad` instead
    return outimage

In [ ]:
# Test your function on the image.
ellsberg_fogged = smoothfog( ellsberg,( 200,300,100 ) )
plt.imshow( ellsberg_fogged,cmap="gray" )
plt.show()

The thin white line at the radius of the circle occurs because the range `i-rad:i+rad` becomes empty at the limit.  To eliminate it, simply skip those lines thus:

    if rad == 0: continue  # after calculating `rad`, before setting `outimage`

If implemented correctly above, then this should work in color as well:

In [ ]:
assange_rgb = imread( 'assange.png' )
plt.imshow( assange_rgb )
plt.show()

assange_smoothed = smoothfog( assange_rgb,( 200,400,150 ) )
plt.imshow( assange_smoothed )
plt.show()

Interestingly, since everything is averaged the filtered portion becomes greyscale.  To fix this, we have to average over a single dimension at a time.  Copy `smoothfog` from above and change the `mean()` statement to `mean( axis=0 ).mean( axis=0 )` (_i.e._, chomp the first axis twice when averaging).

In [ ]:
# Place your modified `smoothfog` here.

In [ ]:
assange_rgb = imread( 'assange.png' )
plt.imshow( assange_rgb )
plt.show()

assange_smoothed = smoothfog( assange_rgb,( 200,400,150 ) )
plt.imshow( assange_smoothed )
plt.show()

This concludes our lab on the mechanics of images and image manipulation using Python.  Most programming languages, including MATLAB, adopt a similar representation when working with images.